## 1. 利用MATLAB 软件编程实现追赶法求解三对角矩阵的算法，并考虑梯形电阻电路问题（图略），已知电路电流${i_1,i_2,\cdots,i_8}$满足下列线性方程组
$$\begin{matrix}
 &2i_1  &-2i_2 &  & & & & &  =V/R\\
 &-2i_1 + &5i_2 &-2i_3& & & & & =0\\
 &-2i_2+ &5i_3 &-2i_4& & & & &=0\\
 & &-2i_3+ &5i_4 &-2i_5& & & &=0\\
 & & &-2i_4 &+5i_5 &-2i_6& & &=0\\
 & & & &-2i_5 &+5i_6 &-2i_7& &=0\\
 & & & & &-2i_6 &+5i_7 &-2i_8&=0\\
 & & & & & &-2i_7 &+5i_8&=0\\
\end{matrix}
$$

## 设V=220V,R=27$\Omega$，求解方程

## 解：三对角矩阵使用追赶法,进行LU分解，再求解方程

### Code

In [1]:
using LinearAlgebra
V=220
Ω=27
A=Tridiagonal([-2,-2,-2,-2,-2,-2,-2], [2,5,5,5,5,5,5,5],[-2,-2,-2,-2,-2,-2,-2])
B=[V/Ω 0 0 0 0 0 0 0]'
A

8×8 Tridiagonal{Int64,Array{Int64,1}}:
  2  -2   ⋅   ⋅   ⋅   ⋅   ⋅   ⋅
 -2   5  -2   ⋅   ⋅   ⋅   ⋅   ⋅
  ⋅  -2   5  -2   ⋅   ⋅   ⋅   ⋅
  ⋅   ⋅  -2   5  -2   ⋅   ⋅   ⋅
  ⋅   ⋅   ⋅  -2   5  -2   ⋅   ⋅
  ⋅   ⋅   ⋅   ⋅  -2   5  -2   ⋅
  ⋅   ⋅   ⋅   ⋅   ⋅  -2   5  -2
  ⋅   ⋅   ⋅   ⋅   ⋅   ⋅  -2   5

In [2]:
function MyTriDiagDecomp(A::Tridiagonal,d)
    a=diag(A,-1)*1.0#Otherwise default type is int
    b=diag(A,0)*1.0
    c=diag(A,1)*1.0
    n,_=size(A)
    #Decompose A, A=LU
    L=Matrix{Float64}(I,size(A))
    U=diagm(1 => c,0=>b*0)
    U[1,1]=b[1]
    for i=2:n
        L[i,i-1]=a[i-1]/U[i-1,i-1]
        U[i,i]=b[i]-L[i,i-1]*c[i-1]
    end
    #Solve Ly=d
    y=copy(d)
    for k=2:n
        y[k]=d[k]-L[k,k-1]*y[k-1]
    end
    #Solve Lx=y
    x=copy(y)
    x[n]=y[n]/U[n,n]
    for k=n-1:-1:1
        x[k]=(y[k]-c[k]*x[k+1])/U[k,k]
    end
    
    return L,U,x
end

MyTriDiagDecomp (generic function with 1 method)

In [3]:
#求解
@time L,U,x=MyTriDiagDecomp(A,B);
x

  0.551478 seconds (1.20 M allocations: 60.765 MiB, 5.30% gc time)


8×1 Array{Float64,2}:
 8.147775166909105  
 4.07370109283503   
 2.0364775651784712 
 1.0174928201111484 
 0.5072544850993995 
 0.25064339263735036
 0.11935399649397638
 0.04774159859759055

In [4]:
#验算，结果正确
A*x

8×1 Array{Float64,2}:
  8.148148148148149     
 -8.881784197001252e-16 
 -1.3322676295501878e-15
  6.661338147750939e-16 
  0.0                   
  0.0                   
  6.938893903907228e-17 
  0.0                   

### 解得 $$\begin{bmatrix}
i_1 \\
i_2\\
i_3\\
i_4\\
i_5\\
i_6\\
i_7\\
i_8\\
\end{bmatrix}=\begin{bmatrix}
 8.147775166909105\\  
 4.07370109283503\\   
 2.0364775651784712\\ 
 1.0174928201111484 \\
 0.5072544850993995 \\
 0.25064339263735036\\
 0.11935399649397638\\
 0.04774159859759055\\
\end{bmatrix}
$$

In [5]:
#Tips
?diagm
?Tridiagonal

LoadError: syntax: invalid identifier name "?"

### 已知Wilson矩阵 $$A=\begin{bmatrix}
10 & 7&8&7\\
7 & 5&6&5\\
8 & 6&10&9\\
7 & 5&9&10
\end{bmatrix}
$$
### 右端项 $b=\begin{bmatrix} 32 &23& 33& 31\end{bmatrix}^T$ , 则有精确解$x=\begin{bmatrix}1 &1& 1&\end{bmatrix}^T$,
### 用MATLAB(这里用Julia)中库函数

#### (1)求det(A)，cond(A),以及A的所有特征值（Julia中基本同名）
#### (2)令$$A+δA=\begin{bmatrix}
10 & 7.2&8.1&6.9\\
7.08 & 5.07&6.02&5\\
8.2 & 5.89&9.96&9.01\\
6.98 & 5.04&8.97&9.98
\end{bmatrix}$$,
#### 解扰动方程(A+δA)(x+δx)=b,并求出向量$δx和||δx||_2$。从理论结果式
$$\frac{||δx||}{||x||}\leq \frac{cond(A)\frac{||δA||_2}{||A||}}{1-cond(A)\frac{||δA||}{||A||}}$$
#### 与实际计算结果两方面分析方程组$Ax=b$的解的相对误差$\frac{||δx||_2}{||x||_2}$ 与A的相对误差$\frac{||δA||_2}{||A||_2}$ 的关系。
#### (3)利用函数rand(n) 重新产生扰动矩阵，使其各元素绝对值不超过$\frac{1}{2}\times 10^{-4}$，重复(2)
##### (注，（2）中理论式只在||δA||||δA||<1 时成立，否则少个负号，应用时我们在右边加了绝对值)

### 解:(1)

In [6]:
A=[10 7 8 7;7 5 6 5;8 6 10 9;7 5 9 10]*1.0
A

4×4 Array{Float64,2}:
 10.0  7.0   8.0   7.0
  7.0  5.0   6.0   5.0
  8.0  6.0  10.0   9.0
  7.0  5.0   9.0  10.0

In [7]:
det(A)

0.9999999999999869

In [8]:
cond(A,2)

2984.0927016757555

### 所有特征值

In [9]:
#?eigvals
#A的特征值
eigvals(A)

4-element Array{Float64,1}:
  0.01015004839789311
  0.843107149855033  
  3.8580574559449485 
 30.288685345802122  

### (2)

In [10]:
A_dis=[10 7.2 8.1 6.9;
7.08 5.07 6.02 5;
8.2 5.89 9.96 9.01;
6.98 5.04 8.97 9.98]
δA=A_dis-A

4×4 Array{Float64,2}:
  0.0    0.2    0.1   -0.1 
  0.08   0.07   0.02   0.0 
  0.2   -0.11  -0.04   0.01
 -0.02   0.04  -0.03  -0.02

In [11]:
b=[32;23;33;31]
x=A\b
x_dis=A_dis\b

4-element Array{Float64,1}:
 0.007731065030371553
 2.3116768552345355  
 1.021050697907897   
 1.0156674404280672  

In [12]:
δx=x_dis-x

4-element Array{Float64,1}:
 -0.9922689349697061  
  1.3116768552346643  
  0.021050697907863736
  0.01566744042808721 

### $||δA||_2,||A||_2,||x||_2$ 计算如下

In [13]:
norm(δA,2),norm(A,2),norm(x,2)

(0.3588871688985267, 30.54504869860253, 1.9999999999999811)

### 根据理论估算式（修正后的） $$\frac{||δx||}{||x||}\leq \left|\frac{cond(A)\frac{||δA||}{||A||}}{1-cond(A)\frac{||δA||}{||A||}}\right|$$

### 得到左端$\frac{||δx||}{||x||}$的上界为下值

In [14]:
abs(cond(A)*norm(δA,2)/norm(A,2)
    /(1-cond(A)*norm(δA,2)/norm(A,2)))

1.0293587346664632

### $||δx||_2$ 实际计算如下

In [15]:
sqrt(δx'δx)

1.6449262635255688

### 得 $\frac{||δx||}{||x||}$ 实际值如下

In [16]:
sqrt(δx'δx)/norm(x,2)

0.8224631317627922

### 符合理论预期

### (3) rand产成均匀分布在[0,1] 之间的扰动，因此我们采用如下的方式生成扰动，确保各元素绝对值不越界。

In [17]:
#δA_new=
δA_new=reshape(rand(length(A)),4,4)*0.5*1e-4

4×4 Array{Float64,2}:
 3.30717e-5  1.13308e-5  4.70317e-6  1.5767e-5 
 2.77079e-5  3.78465e-6  1.92904e-5  3.30343e-5
 1.66354e-6  6.0902e-6   3.51932e-5  4.55776e-5
 4.88181e-5  8.03909e-6  6.58047e-6  4.19648e-5

### 根据理论估算式

### 得到左端$\frac{||δx_{new}||}{||x||}$的上界为下值

In [18]:
abs(cond(A)*norm(δA_new,2)/norm(A,2)
    /(1-cond(A)*norm(δA_new,2)/norm(A,2)))

0.01051573545329872

In [19]:
A_new=A+δA_new
δx_new=A_new\b-x

4-element Array{Float64,1}:
  0.0015630769356393959 
 -0.0025909339575642676 
  0.0006502920947653035 
 -0.00039449432620153324

### $||δx_{new}||_2$ 实际计算如下

In [20]:
sqrt(δx_new'δx_new)

0.003120040682604464

### 得 $\frac{||δx_{new}||}{||x||}$ 实际值如下

In [21]:
sqrt(δx_new'δx_new)/norm(x,2)

0.0015600203413022467

### 符合理论预期